import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc
from pyspark.sql.functions import when

create local SparkSession

In [2]:
import findspark
findspark.init()
spark = SparkSession.builder.appName("task").getOrCreate()

read csv with inferschema

In [3]:
%%time
df_salaries = (spark.read.format('csv')
              .option("header", "true") 
              .option("inferSchema", "true")  
              .load("ds_salaries.csv"))

CPU times: user 1.79 ms, sys: 2.83 ms, total: 4.62 ms
Wall time: 4.12 s


read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [5]:
%%time
df_salaries = (spark.read.format('csv')
              .option("header", "true") 
              .option("inferSchema", "true")  
              .load("ds_salaries.csv"))

CPU times: user 0 ns, sys: 2.71 ms, total: 2.71 ms
Wall time: 321 ms


write schema of scv on screen

In [7]:
%%time
df_salaries.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)

CPU times: user 602 µs, sys: 1.06 ms, total: 1.67 ms
Wall time: 2.66 ms


create schema of this scv

In [3]:
# salaries_schema = df_salaries.schema
salaries_schema = StructType([StructField('_c0', IntegerType(), True), 
                              StructField('work_year', IntegerType(), True), 
                              StructField('experience_level', StringType(), True), 
                              StructField('employment_type', StringType(), True), 
                              StructField('job_title', StringType(), True), 
                              StructField('salary', IntegerType(), True), 
                              StructField('salary_currency', StringType(), True), 
                              StructField('salary_in_usd', IntegerType(), True), 
                              StructField('employee_residence', StringType(), True), 
                              StructField('remote_ratio', IntegerType(), True), 
                              StructField('company_location', StringType(), True), 
                              StructField('company_size', StringType(), True)])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [4]:
%%time
df_salaries = (spark.read.format('csv')
              .option('header', 'true')
              .schema(salaries_schema)
              .load("ds_salaries.csv"))

CPU times: user 1.41 ms, sys: 2.8 ms, total: 4.21 ms
Wall time: 1.56 s


this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [9]:
%%time
df_salaries.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)

CPU times: user 446 µs, sys: 740 µs, total: 1.19 ms
Wall time: 1.27 ms


now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [6]:
df_salaries.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [7]:
df_salaries.toPandas()

,_c0,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [8]:
df_job_titles = (df_salaries.select('job_title')
               .distinct()
               .orderBy('job_title')
               .alias('job_titles'))

print all rows from df_job_titles without truncating jobs

In [9]:
df_job_titles.show(n=df_job_titles.count(), truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|AI Scientist                            |
|Analytics Engineer                      |
|Applied Data Scientist                  |
|Applied Machine Learning Scientist      |
|BI Data Analyst                         |
|Big Data Architect                      |
|Big Data Engineer                       |
|Business Data Analyst                   |
|Cloud Data Engineer                     |
|Computer Vision Engineer                |
|Computer Vision Software Engineer       |
|Data Analyst                            |
|Data Analytics Engineer                 |
|Data Analytics Lead                     |
|Data Analytics Manager                  |
|Data Architect                          |
|Data Engineer                           |
|Data Engineering Manager                |
|Data Science Consultant                 |
|Data Scien

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [10]:
df_analytic = (df_salaries.groupBy('job_title')
              .agg(max('salary_in_usd').alias('max_salary'),
                   avg('salary_in_usd').alias('avg_salary'),
                   min('salary_in_usd').alias('min_salary'))
              .orderBy('job_title'))

print all rows from df_analytic without trancating jobs

In [11]:
df_analytic.show(n=df_analytic.count(), truncate=False)

+----------------------------------------+----------+------------------+----------+
|job_title                               |max_salary|avg_salary        |min_salary|
+----------------------------------------+----------+------------------+----------+
|3D Computer Vision Researcher           |5409      |5409.0            |5409      |
|AI Scientist                            |200000    |66135.57142857143 |12000     |
|Analytics Engineer                      |205300    |175000.0          |135000    |
|Applied Data Scientist                  |380000    |175655.0          |54238     |
|Applied Machine Learning Scientist      |423000    |142068.75         |31875     |
|BI Data Analyst                         |150000    |74755.16666666667 |9272      |
|Big Data Architect                      |99703     |99703.0           |99703     |
|Big Data Engineer                       |114047    |51974.0           |5882      |
|Business Data Analyst                   |135000    |76691.2           |1844

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [12]:
df_analytic = df_analytic.withColumn('row_id',row_number().over(
    Window.orderBy(desc('avg_salary'))
))

print all data from df_analytic

In [13]:
df_analytic.show(n=df_analytic.count(),truncate=False)

+----------------------------------------+----------+------------------+----------+------+
|job_title                               |max_salary|avg_salary        |min_salary|row_id|
+----------------------------------------+----------+------------------+----------+------+
|Data Analytics Lead                     |405000    |405000.0          |405000    |1     |
|Principal Data Engineer                 |600000    |328333.3333333333 |185000    |2     |
|Financial Data Analyst                  |450000    |275000.0          |100000    |3     |
|Principal Data Scientist                |416000    |215242.42857142858|148261    |4     |
|Director of Data Science                |325000    |195074.0          |130026    |5     |
|Data Architect                          |266400    |177873.9090909091 |90700     |6     |
|Applied Data Scientist                  |380000    |175655.0          |54238     |7     |
|Analytics Engineer                      |205300    |175000.0          |135000    |8     |

it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [14]:
df_analytic = df_analytic.select('row_id', 'job_title', 'max_salary', 'avg_salary', 'min_salary')

print df_analytic now

In [15]:
df_analytic.show(n=df_analytic.count(), truncate=False)

+------+----------------------------------------+----------+------------------+----------+
|row_id|job_title                               |max_salary|avg_salary        |min_salary|
+------+----------------------------------------+----------+------------------+----------+
|1     |Data Analytics Lead                     |405000    |405000.0          |405000    |
|2     |Principal Data Engineer                 |600000    |328333.3333333333 |185000    |
|3     |Financial Data Analyst                  |450000    |275000.0          |100000    |
|4     |Principal Data Scientist                |416000    |215242.42857142858|148261    |
|5     |Director of Data Science                |325000    |195074.0          |130026    |
|6     |Data Architect                          |266400    |177873.9090909091 |90700     |
|7     |Applied Data Scientist                  |380000    |175655.0          |54238     |
|8     |Analytics Engineer                      |205300    |175000.0          |135000    |

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [24]:
df_exp_lvl = (df_salaries.groupBy('experience_level')
              .agg(max('salary_in_usd').alias('biggest_salary'))
              .orderBy('biggest_salary'))

print here df_exp_lvl

In [25]:
df_exp_lvl.show(truncate=False)

+----------------+--------------+
|experience_level|biggest_salary|
+----------------+--------------+
|EN              |250000        |
|SE              |412000        |
|MI              |450000        |
|EX              |600000        |
+----------------+--------------+



create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [121]:
df_best = (df_salaries.join(df_exp_lvl, (df_salaries.salary_in_usd == df_exp_lvl.biggest_salary) & (df_salaries.experience_level ==  df_exp_lvl.experience_level)                           )
           .drop(df_salaries["experience_level"])  
          )
df_best = df_best.select(df_best._c0.alias('id'), df_best.experience_level, df_best.biggest_salary, df_salaries.employee_residence)

print df_best

In [122]:
print(f'Number of rows: {df_best.count()}')
df_best.show(n=df_best.count())

Number of rows: 5
+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 33|              MI|        450000|                US|
| 37|              EN|        250000|                US|
| 63|              SE|        412000|                US|
| 97|              MI|        450000|                US|
|252|              EX|        600000|                US|
+---+----------------+--------------+------------------+



drop duplicates if exist by experience_level

In [130]:
# n/a, made on previous step
df_best = df_best.dropDuplicates(['experience_level', 'biggest_salary', 'employee_residence'])

print df_best

In [131]:
df_best.show(n=df_best.count())

+---+----------------+--------------+------------------+
| id|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 33|              MI|        450000|                US|
| 63|              SE|        412000|                US|
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
+---+----------------+--------------+------------------+



create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [132]:
df_new_best = df_best.select(when(df_best.experience_level == 'MI', 'middle') 
                            .when(df_best.experience_level == 'SE', 'senior')
                            .alias('experience_level')
                            , df_best.biggest_salary, df_best.employee_residence
                            )

print df_new_best

In [133]:
df_new_best.show(n=df_new_best.count())

+----------------+--------------+------------------+
|experience_level|biggest_salary|employee_residence|
+----------------+--------------+------------------+
|          middle|        450000|                US|
|          senior|        412000|                US|
|            NULL|        250000|                US|
|            NULL|        600000|                US|
+----------------+--------------+------------------+



write df_new_best like 1.csv and load then it to df_final

In [140]:
df_new_best.toPandas().to_csv('./1.csv', header='True')
df_final = (spark.read.format('csv')
            .option('header', 'true')
            .option('inferSchema', 'true')
            .load('1.csv')
           )

print df_final

In [141]:
df_final.show()

+---+----------------+--------------+------------------+
|_c0|experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
|  0|          middle|        450000|                US|
|  1|          senior|        412000|                US|
|  2|            NULL|        250000|                US|
|  3|            NULL|        600000|                US|
+---+----------------+--------------+------------------+



filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [143]:
df_final = df_final.filter(~df_final.experience_level.isNull())
df_final = df_final.join(df_salaries, (df_final.biggest_salary == df_salaries.salary_in_usd) & (df_final.employee_residence == df_salaries.employee_residence))

print df_final

In [145]:
df_final.show(truncate=False)

+---+----------------+--------------+------------------+---+---------+----------------+---------------+----------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|_c0|experience_level|biggest_salary|employee_residence|_c0|work_year|experience_level|employment_type|job_title             |salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+----------------+--------------+------------------+---+---------+----------------+---------------+----------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|0  |middle          |450000        |US                |33 |2020     |MI              |FT             |Research Scientist    |450000|USD            |450000       |US                |0           |US              |M           |
|1  |senior          |412000        |US                |63 |2020     |SE              |FT       

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [165]:
biggest_salary_in_usd = df_final.select(max('salary_in_usd')).collect()[0][0]
print(f'The biggest salary in usd: {biggest_salary_in_usd}')

The biggest salary in usd: 450000


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system

In [ ]:
spark.stop()